# LangGraph Tutorials


In [3]:
import functools
import operator
from typing import Annotated, Sequence, TypedDict

from dotenv import load_dotenv
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain.tools import tool
from langchain_community.chat_models.litellm import ChatLiteLLM
from langchain_community.tools.ddg_search.tool import DuckDuckGoSearchRun
from langchain_core.messages import BaseMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langgraph.graph import END, StateGraph

#

load_dotenv()

True

In [5]:
llm = ChatOpenAI(model="gpt-4-turbo")

In [6]:
from langchain_core.messages import (
    HumanMessage,
    SystemMessage,
)


@tool("web_search")
def web_search(query: str) -> str:
    """Search with Duck Duck Go by a query"""
    search = DuckDuckGoSearchRun()
    return search.run(query)


@tool("twitter_writer")
def write_tweet(content: str) -> str:
    """Based a piece of content, write a tweet."""
    chat = ChatLiteLLM(model="groq/Llama3-70b-8192")
    messages = [
        SystemMessage(
            content="You are a Twitter account operator."
            " You are responsible for writing a tweet based on the content given."
            " You should follow the Twitter policy and make sure each tweet has no more than 140 characters."
        ),
        HumanMessage(content=content),
    ]
    response = chat(messages)
    return response.content

In [7]:
class AgentState(TypedDict):
    # The annotation tells the graph that new messages will always
    # be added to the current states
    messages: Annotated[Sequence[BaseMessage], operator.add]
    # The 'next' field indicates where to route to next
    next: str

In [8]:
def create_agent(llm: ChatLiteLLM, tools: list, system_prompt: str):
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                system_prompt,
            ),
            MessagesPlaceholder(variable_name="messages"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )
    agent = create_openai_tools_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools)
    return executor


def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {"messages": [HumanMessage(content=result["output"], name=name)]}

In [9]:
members = ["Search_Engine", "Twitter_Writer"]
system_prompt = (
    "You are a supervisor tasked with managing a conversation between the"
    " following workers:  {members}. Given the following user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH."
)

options = ["FINISH"] + members
# Using openai function calling can make output parsing easier for us
function_def = {
    "name": "route",
    "description": "Select the next role.",
    "parameters": {
        "title": "routeSchema",
        "type": "object",
        "properties": {
            "next": {
                "title": "Next",
                "anyOf": [
                    {"enum": options},
                ],
            }
        },
        "required": ["next"],
    },
}
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        (
            "system",
            "Given the conversation above, who should act next?"
            " Or should we FINISH? Select one of: {options}",
        ),
    ]
).partial(options=str(options), members=", ".join(members))

supervisor_chain = (
    prompt
    | llm.bind_functions(functions=[function_def], function_call="route")
    | JsonOutputFunctionsParser()
)

In [10]:
search_engine_agent = create_agent(llm, [web_search], "You are a web search engine.")
search_engine_node = functools.partial(
    agent_node, agent=search_engine_agent, name="Search_Engine"
)

twitter_operator_agent = create_agent(
    llm,
    [write_tweet],
    "You are responsible for writing a tweet based on the content given.",
)
twitter_operator_node = functools.partial(
    agent_node, agent=twitter_operator_agent, name="Twitter_Writer"
)

workflow = StateGraph(AgentState)
workflow.add_node("Search_Engine", search_engine_node)
workflow.add_node("Twitter_Writer", twitter_operator_node)
workflow.add_node("supervisor", supervisor_chain)

In [11]:
for member in members:
    workflow.add_edge(member, "supervisor")

conditional_map = {k: k for k in members}
conditional_map["FINISH"] = END
workflow.add_conditional_edges("supervisor", lambda x: x["next"], conditional_map)

workflow.set_entry_point("supervisor")

graph = workflow.compile()

In [12]:
for s in graph.stream(
    {"messages": [HumanMessage(content="Write a tweet about LangChain news")]}
):
    if "__end__" not in s:
        print(s)
        print("----")

{'supervisor': {'next': 'Twitter_Writer'}}
----


/Users/dothomps/.pyenv/versions/3.11.9/envs/lg/lib/python3.11/site-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_name" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/Users/dothomps/.pyenv/versions/3.11.9/envs/lg/lib/python3.11/site-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_info" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/Users/dothomps/.pyenv/versions/3.11.9/envs/lg/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'Twitter_Writer': {'messages': [HumanMessage(content='"Exciting update from LangChain! Enhanced AI integration for devs makes building AI-powered apps easier & efficient! Check out new features & start innovating! #LangChain #AI #TechUpdate"', name='Twitter_Writer')]}}
----
{'supervisor': {'next': 'FINISH'}}
----
